Credits to professor Mirko Stojiljković for writing this great [article](https://realpython.com/linear-programming-python/)

---



# Installing puLP

In [ ]:
%%capture
!pip install puLP
from pulp import *

# Optional : use GLPK as the solver (default is CPLEX)

Download GLPK and compile it. You can skip this if you've already downloaded *it*

In [ ]:
%%capture
!wget -qO- ftp://ftp.gnu.org/gnu/glpk/glpk-4.65.tar.gz | tar -z -xf- ; cd glpk-4.65; ./configure; make --jobs=4

This is the solver path for Colab, if you're using something else it will be different

In [ ]:
path_to_glpk = r'/content/glpk-4.65/examples/glpsol'
glpk_solver = GLPK_CMD(path=path_to_glpk)

# Otherwise

In [ ]:
def print_solution(model):
    '''
    Prints solution nicely!
    '''
    print(f"status: {model.status}, {LpStatus[model.status]}")
    print(f"objective: {model.objective.value()}")
    for var in model.variables():
        print(f"{var.name}: {var.value()}")



---



Lecture 1 problem

In [ ]:
model = LpProblem('ex1', LpMaximize)

# def decision variables
x = {i: LpVariable(name=f'x({i})', lowBound=0) for i in {1, 2}}

# def objective
model += lpSum(x.values())
# def constraints

model += (2 * x[1] + x[2] <= 11, 'supplyX')
model += (x[1] + 3 * x[2] <= 18, 'supplyY')
model += (x[1] <= 4, 'demandA')

print(model)

status = model.solve(solver=glpk_solver)
# if you installed glpk, otherwise model.solve()

print_solution(model)

ex1:
MAXIMIZE
1*x(1) + 1*x(2) + 0
SUBJECT TO
supplyX: 2 x(1) + x(2) <= 11

supplyY: x(1) + 3 x(2) <= 18

demandA: x(1) <= 4

VARIABLES
x(1) Continuous
x(2) Continuous

status: 1, Optimal
objective: 8.0
x(1): 3.0
x(2): 5.0


Tutorial 1 Exercise 1

In [ ]:
tut_model_1 = LpProblem('tut1', LpMaximize)
# decision variables
x = {i: LpVariable(name=f'x({i})', lowBound=0) for i in range(1,5)}
tut_model_1 += 2 * x[1] + x[2]
# constraints
tut_model_1 += x[1] - 4 * x[2] + x[3] <= 1
tut_model_1 += x[1] + 5 * x[2] - x[4] <= 3

print(tut_model_1)
tut_model_1.solve(solver=glpk_solver)
print_solution(tut_model_1)

tut1:
MAXIMIZE
2*x(1) + 1*x(2) + 0
SUBJECT TO
_C1: x(1) - 4 x(2) + x(3) <= 1

_C2: x(1) + 5 x(2) - x(4) <= 3

VARIABLES
x(1) Continuous
x(2) Continuous
x(3) Continuous
x(4) Continuous

status: -3, Undefined
objective: 0.0
x(1): 0.0
x(2): 0.0
x(3): 0.0
x(4): 0.0


Tutorial 1 Exercise 2.1

In [ ]:
tut_model_2 = LpProblem('tut2', LpMinimize)
# decision variables
x = {(i, j): LpVariable(name=f'x({i},{j})', lowBound=0)
     for (i, j) in itertools.product({'A', 'B'}, {1, 2, 3})}
# objective
tut_model_2 += (x['A', 1] + 2 * x['A', 2] + x['A', 3] + 2 * x['B', 1] +
                x['B', 2] + 2 * x['B', 3])
# constraints
for i in range(1,4):
    tut_model_2 += x['A', i] + x['B', i] == 2
for i in {'A','B'}:
    tut_model_2 += x[i, 1] + x[i, 2] + x[i, 3] <= 3

print(tut_model_2)
tut_model_2.solve(solver=glpk_solver)
print_solution(tut_model_2)

tut2:
MINIMIZE
1*x(A,1) + 2*x(A,2) + 1*x(A,3) + 2*x(B,1) + 1*x(B,2) + 2*x(B,3) + 0
SUBJECT TO
_C1: x(A,1) + x(B,1) = 2

_C2: x(A,2) + x(B,2) = 2

_C3: x(A,3) + x(B,3) = 2

_C4: x(A,1) + x(A,2) + x(A,3) <= 3

_C5: x(B,1) + x(B,2) + x(B,3) <= 3

VARIABLES
x(A,1) Continuous
x(A,2) Continuous
x(A,3) Continuous
x(B,1) Continuous
x(B,2) Continuous
x(B,3) Continuous

status: 1, Optimal
objective: 7.0
x(A,1): 2.0
x(A,2): 0.0
x(A,3): 1.0
x(B,1): 0.0
x(B,2): 2.0
x(B,3): 1.0


Tutorial 1 Exercise 3

In [ ]:
import numpy as np
a = np.array([[1, -1, 1, 1],
             [3, 1, 2, 2],
             [1, 4, -1, -4],
             [2, 0, -2, 3]])
b = np.array([6, 12, 3, -8])

# actually solving the whole system
print(np.linalg.solve(a, b))

# finding its rank
print(np.linalg.matrix_rank(a))

4

Tutorial 1 Exercise 4

In [ ]:
model_4 = LpProblem('ex4', LpMinimize)
# cost array
c = np.array([500, 480, 460, 460, 460, 460, 480])
# chose to have integer variables, made more sense in this case
x = {i: LpVariable(name=f'x({i})', lowBound=0, cat='Integer') for i in range(1, 8)}
# objective
model_4 += lpSum([c[i] * x[i+1] for i in range(7)]) # pulp is 1-indexed
# constraints
# this part could be less terse
import operator
for i in range(1, 8):
    excl = {i, i%7+1}
    if i==1: # for shifts including Sunday
        rhs, op = 8, operator.eq
    elif i in {6, 7}: # for shifts including Friday or Saturday
        rhs, op = 10, operator.ge
    else:
        rhs, op = 6, operator.ge 
    model_4 += (op(lpSum([v for k, v in x.items() if k not in excl]), rhs))
print(model_4)
model_4.solve(solver=glpk_solver)
print_solution(model_4)

ex4:
MINIMIZE
500*x(1) + 480*x(2) + 460*x(3) + 460*x(4) + 460*x(5) + 460*x(6) + 480*x(7) + 0
SUBJECT TO
_C1: x(3) + x(4) + x(5) + x(6) + x(7) = 8

_C2: x(1) + x(4) + x(5) + x(6) + x(7) >= 6

_C3: x(1) + x(2) + x(5) + x(6) + x(7) >= 6

_C4: x(1) + x(2) + x(3) + x(6) + x(7) >= 6

_C5: x(1) + x(2) + x(3) + x(4) + x(7) >= 6

_C6: x(1) + x(2) + x(3) + x(4) + x(5) >= 10

_C7: x(2) + x(3) + x(4) + x(5) + x(6) >= 10

VARIABLES
0 <= x(1) Integer
0 <= x(2) Integer
0 <= x(3) Integer
0 <= x(4) Integer
0 <= x(5) Integer
0 <= x(6) Integer
0 <= x(7) Integer

status: 1, Optimal
objective: 5120
x(1): 0
x(2): 3
x(3): 2
x(4): 3
x(5): 2
x(6): 1
x(7): 0


Case Study 1

In [ ]:
cs_model = LpProblem('Fertilizer_Resource_Allocation', -1)

x = {1: LpVariable(name='S', lowBound=1000),
     2: LpVariable(name='E', lowBound=500),
     3: LpVariable(name='N', lowBound=0),}
s = {1: LpVariable(name='s1', lowBound=0),
     2: LpVariable(name='s2', lowBound=0),
     3: LpVariable(name='s3', lowBound=0),}

cs_model += 3 * x[1] + 5 * x[2] + 5 * x[3]

cs_model += 0.15 * x[1] + 0.1 * x[2] + 0.05 * x[3] + s[1] == 1000
cs_model += 0.05 * x[1] + 0.1 * x[2] + 0.1 * x[3] + s[2] == 1600
cs_model += 0.1 * x[1] + 0.1 * x[2] + 0.15 * x[3] + s[3] == 2500

print(cs_model)
cs_model.solve(solver=glpk_solver)
print_solution(cs_model)

Fertilizer_Resource_Allocation:
MAXIMIZE
5*E + 5*N + 3*S + 0
SUBJECT TO
_C1: 0.1 E + 0.05 N + 0.15 S + s1 = 1000

_C2: 0.1 E + 0.1 N + 0.05 S + s2 = 1600

_C3: 0.1 E + 0.15 N + 0.1 S + s3 = 2500

VARIABLES
500 <= E Continuous
N Continuous
1000 <= S Continuous
s1 Continuous
s2 Continuous
s3 Continuous

status: 1, Optimal
objective: 80700.0
E: 500.0
N: 14800.0
S: 1400.0
s1: 0.0
s2: 0.0
s3: 90.0
